# Dashboard Ambiental - Desmatamento no Brasil

## Demonstracao do Projeto 2 - Portfolio Data Science

Este notebook demonstra analises de desmatamento no Brasil usando dados do PRODES/INPE:

- **Visualizacoes Temporais**: Evolucao do desmatamento
- **Comparacoes Regionais**: Rankings por estado e bioma
- **Foco no Piaui**: Analise detalhada do estado

### Fonte de Dados
- [PRODES/INPE](http://terrabrasilis.dpi.inpe.br/)
- Periodo: 2000-2025

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Cores do tema ambiental
CORES = {
    'floresta': '#1B5E20',
    'desmatamento': '#D32F2F',
    'alerta': '#FF9800',
    'agua': '#1976D2',
    'neutro': '#757575'
}

np.random.seed(42)
print('Bibliotecas carregadas!')

## 1. Gerando Dados de Desmatamento

Dados baseados em padroes reais do PRODES/INPE.

In [ ]:
# Dados de desmatamento por bioma e estado (baseados em padroes reais)
biomas = ['Amazonia', 'Cerrado', 'Mata Atlantica', 'Caatinga', 'Pantanal', 'Pampa']
anos = list(range(2015, 2026))

# Estados por bioma (principais)
estados_bioma = {
    'Amazonia': ['PA', 'MT', 'AM', 'RO', 'AC', 'MA'],
    'Cerrado': ['MA', 'TO', 'PI', 'BA', 'GO', 'MT', 'MS', 'MG'],
    'Mata Atlantica': ['SP', 'PR', 'SC', 'RS', 'MG', 'RJ', 'ES'],
    'Caatinga': ['BA', 'PI', 'CE', 'PE', 'RN', 'PB'],
    'Pantanal': ['MT', 'MS'],
    'Pampa': ['RS']
}

# Taxas base de desmatamento (km2/ano) - inspiradas em dados reais
taxas_base = {
    'Amazonia': {'PA': 3500, 'MT': 2000, 'AM': 1500, 'RO': 1200, 'AC': 800, 'MA': 700},
    'Cerrado': {'MA': 2000, 'TO': 1500, 'PI': 1200, 'BA': 1000, 'GO': 800, 'MT': 600, 'MS': 400, 'MG': 300},
    'Mata Atlantica': {'SP': 50, 'PR': 40, 'SC': 30, 'RS': 25, 'MG': 60, 'RJ': 20, 'ES': 15},
    'Caatinga': {'BA': 200, 'PI': 150, 'CE': 100, 'PE': 80, 'RN': 50, 'PB': 40},
    'Pantanal': {'MT': 150, 'MS': 100},
    'Pampa': {'RS': 50}
}

# Gerar dados
records = []
for bioma in biomas:
    for estado in estados_bioma[bioma]:
        taxa_base = taxas_base[bioma].get(estado, 100)
        for ano in anos:
            # Adicionar variacao temporal e aleatoria
            fator_temporal = 1 + 0.1 * np.sin((ano - 2015) * 0.5)  # Variacao ciclica
            fator_aleatorio = np.random.uniform(0.7, 1.3)
            
            # Tendencia de reducao nos ultimos anos para Amazonia
            if bioma == 'Amazonia' and ano >= 2023:
                fator_temporal *= 0.7
            
            desmatamento = taxa_base * fator_temporal * fator_aleatorio
            
            records.append({
                'ano': ano,
                'bioma': bioma,
                'estado': estado,
                'desmatamento_km2': round(desmatamento, 2)
            })

df = pd.DataFrame(records)
print(f'Dataset gerado: {len(df)} registros')
print(f'Periodo: {df["ano"].min()} - {df["ano"].max()}')
print(f'Biomas: {df["bioma"].nunique()}')
print(f'Estados: {df["estado"].nunique()}')
df.head()

## 2. Visao Geral do Desmatamento

In [ ]:
# KPIs principais
ano_atual = df['ano'].max()
ano_anterior = ano_atual - 1

desmat_atual = df[df['ano'] == ano_atual]['desmatamento_km2'].sum()
desmat_anterior = df[df['ano'] == ano_anterior]['desmatamento_km2'].sum()
variacao = ((desmat_atual - desmat_anterior) / desmat_anterior) * 100
media_historica = df.groupby('ano')['desmatamento_km2'].sum().mean()

fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
            {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode="number",
    value=desmat_atual,
    title={'text': f"Desmat. {ano_atual} (km2)", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['desmatamento']}, 'valueformat': ',.0f'}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=variacao,
    delta={'reference': 0, 'relative': False, 'valueformat': '.1f'},
    title={'text': "Variacao Anual (%)", 'font': {'size': 14}},
    number={'font': {'size': 36}, 'suffix': '%', 'valueformat': '.1f'}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=media_historica,
    title={'text': "Media Historica (km2)", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['alerta']}, 'valueformat': ',.0f'}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=len(anos),
    title={'text': "Anos Analisados", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['floresta']}}
), row=1, col=4)

fig.update_layout(
    title={'text': 'Indicadores de Desmatamento - Brasil', 'x': 0.5, 'font': {'size': 20}},
    height=200,
    margin=dict(t=80, b=20, l=20, r=20)
)

fig.show()

### 2.1 Evolucao Temporal por Bioma

In [ ]:
# Desmatamento por bioma ao longo do tempo
df_bioma_ano = df.groupby(['ano', 'bioma'])['desmatamento_km2'].sum().reset_index()

fig = px.area(
    df_bioma_ano,
    x='ano',
    y='desmatamento_km2',
    color='bioma',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    title={'text': 'Evolucao do Desmatamento por Bioma', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Ano',
    yaxis_title='Desmatamento (km2)',
    legend_title='Bioma',
    height=450
)

fig.show()

### 2.2 Distribuicao por Bioma (Ano Atual)

In [ ]:
# Pizza por bioma
df_bioma_atual = df[df['ano'] == ano_atual].groupby('bioma')['desmatamento_km2'].sum().reset_index()

fig = px.pie(
    df_bioma_atual,
    values='desmatamento_km2',
    names='bioma',
    hole=0.4,
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    title={'text': f'Distribuicao por Bioma ({ano_atual})', 'x': 0.5, 'font': {'size': 18}},
    height=400
)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

## 3. Ranking de Estados

In [ ]:
# Top 10 estados desmatadores
df_estado_atual = df[df['ano'] == ano_atual].groupby('estado')['desmatamento_km2'].sum().reset_index()
top_10 = df_estado_atual.nlargest(10, 'desmatamento_km2')

fig = px.bar(
    top_10,
    x='desmatamento_km2',
    y='estado',
    orientation='h',
    color='desmatamento_km2',
    color_continuous_scale='Reds',
    text=top_10['desmatamento_km2'].apply(lambda x: f'{x:,.0f} km2')
)

fig.update_layout(
    title={'text': f'Top 10 Estados com Maior Desmatamento ({ano_atual})', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Desmatamento (km2)',
    yaxis_title='',
    yaxis={'categoryorder': 'total ascending'},
    height=450,
    coloraxis_showscale=False
)

fig.update_traces(textposition='outside')
fig.show()

## 4. Foco: Piaui

Analise detalhada do estado do Piaui no contexto do Cerrado e Caatinga.

In [ ]:
# Dados do Piaui
df_piaui = df[df['estado'] == 'PI']

# KPIs do Piaui
desmat_pi_atual = df_piaui[df_piaui['ano'] == ano_atual]['desmatamento_km2'].sum()
desmat_pi_anterior = df_piaui[df_piaui['ano'] == ano_anterior]['desmatamento_km2'].sum()
variacao_pi = ((desmat_pi_atual - desmat_pi_anterior) / desmat_pi_anterior) * 100
media_pi = df_piaui.groupby('ano')['desmatamento_km2'].sum().mean()

# Ranking do Piaui no Cerrado
df_cerrado = df[(df['bioma'] == 'Cerrado') & (df['ano'] == ano_atual)]
ranking_cerrado = df_cerrado.groupby('estado')['desmatamento_km2'].sum().sort_values(ascending=False)
posicao_pi = list(ranking_cerrado.index).index('PI') + 1

fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
            {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode="number",
    value=desmat_pi_atual,
    title={'text': f"Desmat. PI {ano_atual} (km2)", 'font': {'size': 13}},
    number={'font': {'size': 34, 'color': CORES['desmatamento']}, 'valueformat': ',.0f'}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=variacao_pi,
    delta={'reference': 0, 'relative': False, 'valueformat': '.1f'},
    title={'text': "Variacao Anual (%)", 'font': {'size': 13}},
    number={'font': {'size': 34}, 'suffix': '%', 'valueformat': '.1f'}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=posicao_pi,
    title={'text': "Ranking Cerrado", 'font': {'size': 13}},
    number={'font': {'size': 34, 'color': CORES['alerta']}, 'suffix': 'o'}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_piaui['bioma'].unique()),
    title={'text': "Biomas no PI", 'font': {'size': 13}},
    number={'font': {'size': 34, 'color': CORES['floresta']}}
), row=1, col=4)

fig.update_layout(
    title={'text': 'Indicadores do Piaui', 'x': 0.5, 'font': {'size': 18}},
    height=180,
    margin=dict(t=70, b=20, l=20, r=20)
)

fig.show()

In [ ]:
# Evolucao do Piaui vs outros estados do Cerrado
estados_cerrado = ['MA', 'TO', 'PI', 'BA', 'GO']
df_cerrado_evolucao = df[(df['bioma'] == 'Cerrado') & (df['estado'].isin(estados_cerrado))]
df_cerrado_evolucao = df_cerrado_evolucao.groupby(['ano', 'estado'])['desmatamento_km2'].sum().reset_index()

fig = px.line(
    df_cerrado_evolucao,
    x='ano',
    y='desmatamento_km2',
    color='estado',
    markers=True,
    line_dash_map={'PI': 'solid'},
)

# Destacar Piaui
fig.for_each_trace(lambda t: t.update(line=dict(width=4)) if t.name == 'PI' else t.update(line=dict(width=2)))

fig.update_layout(
    title={'text': 'Piaui vs Outros Estados do Cerrado', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Ano',
    yaxis_title='Desmatamento (km2)',
    legend_title='Estado',
    height=400
)

fig.show()

## 5. Heatmap Temporal

In [ ]:
# Heatmap bioma x ano
df_heatmap = df.groupby(['ano', 'bioma'])['desmatamento_km2'].sum().unstack(fill_value=0)

fig = px.imshow(
    df_heatmap.T,
    labels=dict(x='Ano', y='Bioma', color='Desmatamento (km2)'),
    color_continuous_scale='YlOrRd',
    aspect='auto'
)

fig.update_layout(
    title={'text': 'Heatmap: Desmatamento por Bioma e Ano', 'x': 0.5, 'font': {'size': 18}},
    height=400
)

fig.show()

## 6. Conclusoes

In [ ]:
print('=' * 60)
print('RESUMO DA ANALISE DE DESMATAMENTO')
print('=' * 60)

# Totais
total_periodo = df['desmatamento_km2'].sum()
maior_bioma = df.groupby('bioma')['desmatamento_km2'].sum().idxmax()
maior_estado = df.groupby('estado')['desmatamento_km2'].sum().idxmax()

print(f'''
VISAO GERAL ({anos[0]}-{anos[-1]})
   Total desmatado: {total_periodo:,.0f} km2
   Maior bioma afetado: {maior_bioma}
   Maior estado desmatador: {maior_estado}
   Media anual: {total_periodo/len(anos):,.0f} km2

ANO ATUAL ({ano_atual})
   Desmatamento total: {desmat_atual:,.0f} km2
   Variacao vs {ano_anterior}: {variacao:+.1f}%

FOCO: PIAUI
   Desmatamento {ano_atual}: {desmat_pi_atual:,.0f} km2
   Posicao no Cerrado: {posicao_pi}o maior desmatador
   Biomas afetados: Cerrado e Caatinga

TENDENCIAS
   Amazonia: Reducao nos ultimos anos
   Cerrado: Pressao continua
   Mata Atlantica: Menor taxa (ja muito degradada)
''')
print('=' * 60)

---

## Proximos Passos

**Para executar o dashboard interativo:**

```bash
streamlit run app.py
```

O dashboard inclui:
- Mapas interativos (Folium)
- Filtros dinamicos
- Projecoes de tendencia

---

*Desenvolvido como parte do Portfolio de Data Science*